# Assignment 2
Or *'How I Learned to Start Worrying and Shoot, It's Fluids Again'*

BMI 2005  
Due: 2/13/2019

By: **Ryan Neff**  
ryan.neff@icahn.mssm.edu

## Problem 1

As we discussed in class, simulation of a “real” liquid, such as a box of non-bonded particles at a given temperature and pressure, requires consideration of so-called periodic boundary conditions. In our simulation, we will assume the center of the box is at position `0,0,0` and that the initial vectors describing the extent of the box are equal (e.g., this box is a square!).   

Attached to this homework is a new `XYZ` file defining a box of 200 Lennard-Jones particles. The boundary vector for this box for a reasonable box temperature and pressure (we’ll get there later, don’t worry about this yet...) is **`3.21829795`**. The total length of the box is twice the vector length in each dimension, so this box is **`6.4365959`** in length in each dimension (x, y, and z).   

### Problem 1a

Implement a new total LJ potential function (as you did for problem 4(a) in the previous homework) which includes consideration of a periodic boundary, and compute the total potential for the system in the supplied new `XYZ` file (`lj-0200-liquid.xyz`).   

This will require you to correct the distance between atoms in each dimension. You may in the previous homework have written something like the following, which computes the distance based on the vector norm between the `x`, `y`, and `z` positions of two atoms, and the conditional defining your cutoff distance:

```
def distance(a, b):
    return np.linalg.norm(a - b)

def V_ij(r):
    return 4 *((1/r)**12 - (1/r)**6)

def lj_cut(atoms, r_cut): 
    n_atoms = a.shape[0] 
    LJ_potential = 0.0
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms): 
            r = distance(a[i], a[j])
            if r < r_cut:
                LJ_potential += V_ij(r) 
    return LJ_potential
```

Because you will need to correct the distance between atoms in each dimension, based on the periodicity of the system, this approach will not quite work: you will need to consider the distance in each dimension between the two particles in your double loop. In this case, it is easiest to consider the square euclidian distance, because it allows you to easily accumulate the distance in each dimension and then simply take a square root at the end. E.g.,

$d^2(p,q) = (p_1-q_1)^2 + (p_2-q_2)^2 + ... + (p_i-q_i)^2 + ... + (p_n-q_n)^2$

In pseudowords, a good approach to this is the following:

```
for each atom i in the system
    for each pairwise interaction atom j
        square_distance ← 0.0 
        for each dim
            distance_in_dim ← a[i][dim] - a[j][dim] 
            distance_in_dim -= boundary_length[dim] * \
                round(distance_in_dim / boundary_length[dim]) 
            square_distance += distance_in_dim ** 2
        distance = sqrt(square_distance) 
        if (distance < cutoff...)...
```

Where `distance_in_dim` is modified using the supplied function.

In [53]:
import numpy as np
import re

def read_xyz(input_filename):
    '''read_xyz(input_filename)
    Reads an XYZ file to a numpy float32 array.

    Inputs:
        input_filename
            Filename of XYZ file on disk. 

            File format: 
                line 1: <number of atoms / lines>
                line2: <comment>
                line 3+: <atom type> <x> <y> <z>
    
    Returns: 
        output
            Array of 3D positions of atoms in file. 
    '''

    number_of_atoms, comment, output = None, None, []
    with open(input_filename,"r") as fp:
        for line in fp:
            line = line.strip() #clean input
            if number_of_atoms == None:
                number_of_atoms = int(line)
            elif comment == None:
                comment = line
            else:
                splitline = [float(a) for a in re.split(" +",line)[1:]]
                output.append(splitline)
    output = np.array(output,dtype="float32")
    return output

def V_ij(r):
    '''V_ij(r)
     Computes the potential energy V_ij between two atoms i and j
     with distance r where e and s are set to 1

     Inputs: 
        r (float)
            Distance between two atoms i and j
    
    Outputs:
        V_ij (float)
            Potential energy between two atoms
    '''
    return 4 *((1/r)**12 - (1/r)**6)

def LJ_potential(atoms,boundary_length=[6.4365959]*(3), cutoff=False):
    '''LJ_potential(atoms,boundary_length=[6.4365959]*(3), cutoff=False)
    Calculates the energy potential of the system for a set of atoms 
    based on N-dimensional coordinates, using the LJ potential equation. 
    
    Inputs: 
        atoms
            An array atoms, with each atom being a row of
            n-dimensions with position information.
        boundary_length
            The boundary distance vector over some periodic interaction
            function occurs.
        cutoff
            Whether a cutoff distance exists to consider interactions is set. 
            Either False or a number indicating the cutoff distance.
    
    Returns:
        total_energy (float)
            The total energy of the system.
    '''
    total_energy = 0
    dimensions = range(0,len(atoms[0]))
    for atom1 in range(0,len(atoms)):
        for atom2 in range(atom1+1,len(atoms)):
            square_dist = 0
            for dim in dimensions:
                dist_in_dim = atoms[atom1,dim] - atoms[atom2,dim]
                dist_in_dim -= boundary_length[dim] * round(dist_in_dim/boundary_length[dim]) ##??
                dist_in_dim = dist_in_dim
                square_dist += dist_in_dim**2
            distance = np.sqrt(square_dist)
            if cutoff:
                if distance < cutoff: 
                    total_energy += V_ij(distance)
            else:
                total_energy += V_ij(distance)
    return total_energy

In [56]:
LJ_potential(read_xyz("lj-0200-liquid.xyz"))

-1002.2429537424274

### Problem 1b

Compute the new potential energy for the system using your new function and a
cutoff value of 2.0. Report the value.

In [102]:
LJ_potential(read_xyz("lj-0200-liquid.xyz"), cutoff=2.0)

-862.15524954568468

## Problem 2

We discussed two methods of list construction, the Verlet list, and the cell list, which can be implemented by an elementary hash table, and a linked list, respectively.

<img src="https://i.imgur.com/bA3ogu4.png" width=300>

### Problem 2b

First, we will implement a static Verlet list and rewrite our potential energy function once again to accommodate this new data structure. Constructing the list involves looping over all atom pairs, determining which pairs are within the cutoff + sheath distance, and constructing two arrays: a lookup table, where the location of pair information can be found for each atom, and a value table, where all pair atoms are found. The number of pairwise interactions per atom can be determined purely by the values in the lookup table, by comparing the position in the value table for the current and next atom.

Just like our previous attempt at a new potential function, periodicity must be considered. A pseudoword attempt at the list function might look like, for a cutoff of 2.0 and a sheath of 0.3:

```
cutoff ← 2.0
sheath ← 0.3
cutoff_plus_sheath ← cutoff + sheath
lookup ← empty numpy array of integers, length n_atoms 
value ← unknown length, placeholder array
pair_counter ← 0
for each atom i in the system
    for each pairwise interaction atom j 
        square_distance ← 0.0
        for each dim
            distance_in_dim ← a[i][dim] - a[j][dim] 
            distance_in_dim -= boundary_length[dim] * \
                round(distance / boundary_length[dim]) square_distance += distance_in_dim ** 2
            square_distance += distance
        distance = sqrt(square_distance) 
        if (distance < cutoff_plus_sheath)
            pair_counter ← pair_counter + 1
            value ← push pair index j onto array 
        lookup[i] ← pair_counter
```

In [98]:
def calculate_pairs_verlet(atoms, boundary_length = [6.4365959]*(3), cutoff=2.0, sheath=0.3):
    '''calculate_pairs_verlet(atoms, cutoff=2.0, sheath=0.3)
    Calculates which atoms interact in a set of atoms based on their 
    N-dimensional coordinates. This function implements this via use 
    of a Verlet list.
    
    Inputs: 
        atoms
            A numpy array of atoms, with each atom being a row of
            n-dimensions with position information.
        boundary_length
            The boundary distance box over which some periodic 
            interaction function occurs. Default = [6.4365959]*(3).
        cutoff
            The cutoff distance to consider interactions is set, 
            as a number. Default = 2.0.
        sheath
            The sheath distance within to consider interactions.
            Default = 0.3.
    
    Returns:
        lookup
            lookup table for verlet list
        value
            value table for verlet list
    '''
    total_energy = 0
    dimensions = range(0,len(atoms[0]))
    cutoff_plus_sheath = cutoff + sheath
    lookup = [0]*(len(atoms))
    value = []
    pair_counter = 0
    for atom1 in range(0,len(atoms)):
        lookup[atom1] = pair_counter ## this needs to be initialized in the beginning, not the end
        for atom2 in range(atom1+1,len(atoms)):
            square_dist = 0
            for dim in dimensions:
                dist_in_dim = atoms[atom1,dim] - atoms[atom2,dim]
                dist_in_dim -= boundary_length[dim] * round(dist_in_dim/boundary_length[dim]) ##??
                dist_in_dim = dist_in_dim
                square_dist += dist_in_dim**2
            distance = np.sqrt(square_dist)
            if distance < cutoff_plus_sheath: 
                pair_counter += 1
                value.append(atom2) #why not save a lot of time here?
    return lookup,value

### Problem 2b

After implementing the above pairwise function to generate a Verlet list, implement a total potential energy function that takes as input both arrays in the Verlet list, with appropriate control logic to only include those atoms within the cutoff but not within the sheath. Argue for why the complexity of this function is now O(N).

In [99]:
def LJ_potential_verlet(atoms,lookup_value, boundary_length = [6.4365959]*(3), cutoff=2):
    '''
    Inputs:
        lookup_value
            Tuple of (lookup, value) from calculate_pairs_verlet().
        cutoff
            The cutoff distance to consider interactions is set, 
            as a number. Default = 2.0.
    Returns: 
        total_energy (float)
            The total energy of the system.
    '''
    lookup, value = lookup_value
    total_energy = 0
    dimensions = range(0,len(atoms[0]))
    for atom1 in range(0,len(lookup)):
        start = lookup[atom1]
        end = lookup[atom1+1] if atom1 < (len(lookup)-1) else len(value)
        pair_atoms = value[start:end]
        for atom2 in pair_atoms:
            square_dist = 0
            for dim in dimensions:
                dist_in_dim = atoms[atom1,dim] - atoms[atom2,dim]
                dist_in_dim -= boundary_length[dim] * round(dist_in_dim/boundary_length[dim]) ##??
                dist_in_dim = dist_in_dim
                square_dist += dist_in_dim**2
            distance = np.sqrt(square_dist)
            if distance==0: raise
            if distance < cutoff:
                total_energy += V_ij(distance)
    return total_energy

In [103]:
atoms = read_xyz("lj-0200-liquid.xyz")
LJ_potential_verlet(atoms, calculate_pairs_verlet(atoms))

-862.15524954568468

The complexity of this function is now $O(n)$ because we assume that each atom only has a limited number of interaction pairs (say, $10$ on average) and therefore, we are only iterating over $N*10$ atoms on average, even if the number of atoms becomes very large.

## Problem 3

Now we will tackle the linked cell formulation, which is the first method we will cover to transform this problem into a true linear or logarithmetic problem. We will start with a very simple implementation of the cell list method, simply creating an appropriate set of linked lists, and continue to improve on our implementation with time once we start actually moving atoms in the simulation.

### Problem 3a

Given a fixed box length of 6.4365959 in all directions, and a cutoff of 2.0, what would be an appropriate size for a cell-based domain decomposition of this box? Why? (Hint: we covered this in the 4th week of class).

Answer: We would split up the box into a $3*3*3$ set of cells, so that all of the interactions (and the sheath) will almost all nicely fit within the box. 

### Problem 3b

Create an array of linked lists for a 3x3x3 (27 lists) decomposition of the array. How you index the position of each cell is up to you, but you need to know which area of the box each cell represents! Each node in each of the linked list should contain at a minimum a index value (the index of the atom) and a pointer to next.